Mnist MultiLayerPerceptron

In [28]:
%matplotlib inline
import tensorflow as tf
import tensorflow.contrib.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.examples.tutorials.mnist import input_data
import time

def get_mnist():
    mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)
    return mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

class NNModel():
    '''
    Logistic Regression Model
    '''
    
    def __init__(self, learning_rate = 0.0001, shape = (784, 10), n_hidden = 30, n_hidden_2 = 20 , activation = tf.nn.relu):
        self.activation = activation
        self.learning_rate = learning_rate
        self.shape = shape
        # placeholders
        self.X = tf.placeholder(tf.float32,[None, self.shape[0]], name = 'X')
        self.Y = tf.placeholder(tf.float32, name = 'Y')
        #layers
        self.fc1 = layers.fully_connected(self.X, n_hidden, activation_fn = self.activation)
        self.fc2 = layers.fully_connected(self.fc1, n_hidden_2, activation_fn = self.activation)
        self.Y_hat = layers.fully_connected(self.fc2, shape[1], activation_fn = None)
        #loss and optimization
        #self.loss = tf.reduce_mean(tf.square(self.Y - self.Y_hat, name = 'loss'))
        # cross entropy is better for classification problem
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = self.Y, logits = self.Y_hat, name = 'loss'))
        #self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss)
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        # prediction and accuracy
        self.correct_prediction = tf.equal(tf.argmax(self.Y_hat, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        # for tensorboard
        self.loss_scalar = tf.summary.scalar('cross-entropy', self.loss)
        self.accuracy_scalar = tf.summary.scalar('accuracy', self.accuracy)
        self.merged_summary_op = tf.summary.merge_all()
        # inititialization and session stuff
        self.init_op = tf.global_variables_initializer()
        self.sess = tf.Session()
        
    def fit(self, X, Y, epochs = 100, every = 10, folder = 'NNLog', batch_size = 500, init = True ):
        '''
        optimizes weights
        args:
          X: <numpy.array>, features 
          Y: <numpy.array>, labels
          epochs: <int>, number of epochs
          every: <int>, how often print message with Epoch and Loss values
          folder: <string>, name of folder where to store data for TensorBoard
          batch_size: <int>, size of features array, size of features sample
        '''
        total = []
        if init:
            self.sess.run(self.init_op)
        summary_writer = tf.summary.FileWriter(folder, self.sess.graph)
        x_length = len(X)
        for i in range(epochs):
            batch = np.random.randint(0, x_length - batch_size, 1)[0]
            x_batch = X[batch:batch + batch_size]
            y_batch = Y[batch:batch + batch_size]
            _, l, a = self.sess.run([self.optimizer, self.loss, self.accuracy], feed_dict = {self.X: x_batch, self.Y: y_batch})
            loss, accuracy = self.sess.run([self.loss_scalar, self.accuracy_scalar ], feed_dict = {self.X: x_batch, self.Y: y_batch})
            summary_writer.add_summary(loss, i)
            summary_writer.add_summary(accuracy)
            total.append(l)
            if i % every == 0:
                print('[ {} ] Epoch {} Loss: {} Accuracy {}'.format(time.ctime(), i, l, a))
        return total
    
    def predict(self, X, before_fit = False):
        '''
        return predicted values, 
        
        args:
          X: <numpy.array>, features
          before_fit: <boolean>, change to True if you want to use before calling fit method
        '''
        if before_fit:
            self.sess.run(self.init_op)
        Y_hat = self.sess.run(self.Y_hat, feed_dict = {self.X: X})
        return Y_hat
    
    def show(self, values):
        '''
        plot graph
        
        args:
          values: <list>, list or array of values to be plotted
        '''
        plt.plot(values)
        plt.show()
    
    def close_session(self):
        '''
        closes tensorflow session
        '''
        self.sess.close()
        return True
    
    def close_and_reset(self):
        '''
        closes tensorflow session, clears the default graph stack and resets the global default graph.
        '''
        self.sess.close()
        tf.reset_default_graph()
        return True

In [3]:
X_train, Y_train, X_test, Y_test = get_mnist()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [29]:
nn = NNModel()
losses = nn.fit(X_train, Y_train, epochs = 10000, every = 500, batch_size = 2500)

[ Wed Dec 26 12:20:21 2018 ] Epoch 0 Loss: 2.3153092861175537 Accuracy 0.12080000340938568
[ Wed Dec 26 12:20:37 2018 ] Epoch 500 Loss: 0.6353247165679932 Accuracy 0.8568000197410583
[ Wed Dec 26 12:20:54 2018 ] Epoch 1000 Loss: 0.35971659421920776 Accuracy 0.9020000100135803
[ Wed Dec 26 12:21:10 2018 ] Epoch 1500 Loss: 0.3106103837490082 Accuracy 0.9115999937057495
[ Wed Dec 26 12:21:27 2018 ] Epoch 2000 Loss: 0.2133604735136032 Accuracy 0.9395999908447266
[ Wed Dec 26 12:21:43 2018 ] Epoch 2500 Loss: 0.22712615132331848 Accuracy 0.9359999895095825
[ Wed Dec 26 12:22:00 2018 ] Epoch 3000 Loss: 0.2420654296875 Accuracy 0.9323999881744385
[ Wed Dec 26 12:22:17 2018 ] Epoch 3500 Loss: 0.19117936491966248 Accuracy 0.946399986743927
[ Wed Dec 26 12:22:33 2018 ] Epoch 4000 Loss: 0.170494943857193 Accuracy 0.9488000273704529
[ Wed Dec 26 12:22:50 2018 ] Epoch 4500 Loss: 0.16485324501991272 Accuracy 0.9527999758720398
[ Wed Dec 26 12:23:06 2018 ] Epoch 5000 Loss: 0.14597132802009583 Accuracy